In [1]:
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], ".."))
from modt.modt import MoDT
from modt._initialization import *
from modt.visualization import *
from modt.utility import *

import pickle
from timeit import default_timer as timer

import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import normalize
from sklearn.model_selection import RepeatedKFold

In [2]:
datasets = [
    ["abalone_input.pd","abalone_target.pd"], 
    ["adult_input.pd","adult_target.pd"], # Large
    ["banknote_input.pd","banknote_target.pd"], # Easy
    ["bank_input.pd","bank_target.pd"], # Large
    ["breast_cancer_input.np","breast_cancer_target.np"],
    ["cars_input.pd","cars_target.pd"], 
    ["contraceptive_input.pd","contraceptive_target.pd"], 
    ["generated6_input.np","generated6_target.np"],
    ["hrss_input.pd","hrss_target.pd"], # Large
    ["iris_input.pd","iris_target.pd"],
    ["steel_input.pd","steel_target.pd"],
    ["students_input.pd","students_target.pd"],
    #["sensorless_input.pd","sensorless_target.pd"], # Very Large dataset
]

In [3]:
parameters = {
    "X": None,
    "y": None,
    "n_experts": "to be overwritten",
    "iterations": 100,
    "max_depth": "to be overwritten",
    "init_learning_rate": 110,
    "learning_rate_decay": 0.985,
    "initialization_method": "random",
    "feature_names": None,
    "class_names": None,
    "use_2_dim_gate_based_on": "to be overwritten",
    "use_2_dim_clustering": False,
    "black_box_algorithm": None,
    }
parameters_fit = {
    "optimization_method": "least_squares_linear_regression",
    "early_stopping": True,
    "use_posterior": False,
    }

In [4]:
gate_reduction_methods = [
    "feature_importance_lr_max",
    None
]

In [5]:
n_experts = np.arange(1,11)
n_experts

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [6]:
start = timer()
runs = 5
rows = []
for dataset in datasets:
    print("Starting",dataset[0],"...")
    data_input = pickle.load(open("../datasets/" + dataset[0], "rb"))
    data_target = pickle.load(open("../datasets/" + dataset[1], "rb"))
    
    use_dataframe = False
    if isinstance(data_input, pd.core.frame.DataFrame):
        use_dataframe = True
        
    dict_results = {
        "dataset" : dataset[0].split("_")[0],
        "n_features" : data_input.shape[1]
    } 
    
    for method in gate_reduction_methods:
        parameters["use_2_dim_gate_based_on"] = method    
        for n_expert in n_experts:            
            parameters["n_experts"] = n_expert
            for depth in [1,2]:
                parameters["max_depth"] = depth
                print("Starting",dataset[0],method,"experts:",n_expert,"depth:",depth,"...")

                train_accuracies = []
                val_accuracies = []

                rkf = RepeatedKFold(n_splits=4, n_repeats=runs)
                for train_idx, val_idx in rkf.split(data_input):
                    if use_dataframe:
                        X_temp = data_input.iloc[train_idx]
                        y_temp = data_target.iloc[train_idx]
                        X_temp.reset_index(inplace=True, drop=True)
                        y_temp.reset_index(inplace=True, drop=True)
                    else:
                        X_temp = data_input[train_idx]
                        y_temp = data_target[train_idx]

                    parameters["X"] = X_temp
                    parameters["y"] = y_temp

                    modt = MoDT(**parameters)
                    modt.fit(**parameters_fit)
                    train_accuracies.append(modt.score_internal_disjoint())

                    if use_dataframe:
                        X_temp = data_input.iloc[val_idx]
                        y_temp = data_target.iloc[val_idx]
                        X_temp.reset_index(inplace=True, drop=True)
                        y_temp.reset_index(inplace=True, drop=True)
                    else:
                        X_temp = data_input[val_idx]
                        y_temp = data_target[val_idx]
                    val_accuracies.append(modt.score(X_temp, y_temp))

                train_accuracy = np.mean(train_accuracies)
                val_accuracy = np.mean(val_accuracies)
                train_std = np.std(train_accuracies)
                val_std = np.std(val_accuracies)
                dict_results["train"] = train_accuracy
                dict_results["test"] = val_accuracy
                dict_results["train_std"] = train_std
                dict_results["test_std"] = val_std 

                dict_results["n_experts"] = parameters["n_experts"]
                dict_results["depth"] = parameters["max_depth"]
                dict_results["gate"] = method

                rows.append(dict_results.copy())
    
print("Duration", timer() - start)
df_performance = pd.DataFrame(rows)

Starting generated6_input.np ...
Starting generated6_input.np feature_importance_lr_max experts: 1 depth: 1 ...
Starting generated6_input.np feature_importance_lr_max experts: 1 depth: 2 ...
Starting generated6_input.np feature_importance_lr_max experts: 2 depth: 1 ...
Starting generated6_input.np feature_importance_lr_max experts: 2 depth: 2 ...
Starting generated6_input.np feature_importance_lr_max experts: 3 depth: 1 ...
Starting generated6_input.np feature_importance_lr_max experts: 3 depth: 2 ...
Starting generated6_input.np feature_importance_lr_max experts: 4 depth: 1 ...
Starting generated6_input.np feature_importance_lr_max experts: 4 depth: 2 ...
Starting generated6_input.np feature_importance_lr_max experts: 5 depth: 1 ...
Starting generated6_input.np feature_importance_lr_max experts: 5 depth: 2 ...
Starting generated6_input.np feature_importance_lr_max experts: 6 depth: 1 ...
Starting generated6_input.np feature_importance_lr_max experts: 6 depth: 2 ...
Starting generated6

Starting steel_input.pd ...
Starting steel_input.pd feature_importance_lr_max experts: 1 depth: 1 ...
Starting steel_input.pd feature_importance_lr_max experts: 1 depth: 2 ...
Starting steel_input.pd feature_importance_lr_max experts: 2 depth: 1 ...
Starting steel_input.pd feature_importance_lr_max experts: 2 depth: 2 ...
Starting steel_input.pd feature_importance_lr_max experts: 3 depth: 1 ...
Starting steel_input.pd feature_importance_lr_max experts: 3 depth: 2 ...
Starting steel_input.pd feature_importance_lr_max experts: 4 depth: 1 ...
Starting steel_input.pd feature_importance_lr_max experts: 4 depth: 2 ...
Starting steel_input.pd feature_importance_lr_max experts: 5 depth: 1 ...
Starting steel_input.pd feature_importance_lr_max experts: 5 depth: 2 ...
Starting steel_input.pd feature_importance_lr_max experts: 6 depth: 1 ...
Starting steel_input.pd feature_importance_lr_max experts: 6 depth: 2 ...
Starting steel_input.pd feature_importance_lr_max experts: 7 depth: 1 ...
Starting s

Starting cars_input.pd feature_importance_lr_max experts: 1 depth: 2 ...
Starting cars_input.pd feature_importance_lr_max experts: 2 depth: 1 ...
Starting cars_input.pd feature_importance_lr_max experts: 2 depth: 2 ...
Starting cars_input.pd feature_importance_lr_max experts: 3 depth: 1 ...
Starting cars_input.pd feature_importance_lr_max experts: 3 depth: 2 ...
Starting cars_input.pd feature_importance_lr_max experts: 4 depth: 1 ...
Starting cars_input.pd feature_importance_lr_max experts: 4 depth: 2 ...
Starting cars_input.pd feature_importance_lr_max experts: 5 depth: 1 ...
Starting cars_input.pd feature_importance_lr_max experts: 5 depth: 2 ...
Starting cars_input.pd feature_importance_lr_max experts: 6 depth: 1 ...
Starting cars_input.pd feature_importance_lr_max experts: 6 depth: 2 ...
Starting cars_input.pd feature_importance_lr_max experts: 7 depth: 1 ...
Starting cars_input.pd feature_importance_lr_max experts: 7 depth: 2 ...
Starting cars_input.pd feature_importance_lr_max ex

Starting bank_input.pd feature_importance_lr_max experts: 4 depth: 2 ...
Starting bank_input.pd feature_importance_lr_max experts: 5 depth: 1 ...
Starting bank_input.pd feature_importance_lr_max experts: 5 depth: 2 ...
Starting bank_input.pd feature_importance_lr_max experts: 6 depth: 1 ...
Starting bank_input.pd feature_importance_lr_max experts: 6 depth: 2 ...
Starting bank_input.pd feature_importance_lr_max experts: 7 depth: 1 ...
Starting bank_input.pd feature_importance_lr_max experts: 7 depth: 2 ...
Starting bank_input.pd feature_importance_lr_max experts: 8 depth: 1 ...
Starting bank_input.pd feature_importance_lr_max experts: 8 depth: 2 ...
Starting bank_input.pd feature_importance_lr_max experts: 9 depth: 1 ...
Starting bank_input.pd feature_importance_lr_max experts: 9 depth: 2 ...
Starting bank_input.pd feature_importance_lr_max experts: 10 depth: 1 ...
Starting bank_input.pd feature_importance_lr_max experts: 10 depth: 2 ...
Starting bank_input.pd None experts: 1 depth: 1 .

Starting pdm6_input.pd feature_importance_lr_max experts: 7 depth: 2 ...
Starting pdm6_input.pd feature_importance_lr_max experts: 8 depth: 1 ...
Starting pdm6_input.pd feature_importance_lr_max experts: 8 depth: 2 ...
Starting pdm6_input.pd feature_importance_lr_max experts: 9 depth: 1 ...
Starting pdm6_input.pd feature_importance_lr_max experts: 9 depth: 2 ...
Starting pdm6_input.pd feature_importance_lr_max experts: 10 depth: 1 ...
Starting pdm6_input.pd feature_importance_lr_max experts: 10 depth: 2 ...
Starting pdm6_input.pd None experts: 1 depth: 1 ...
Starting pdm6_input.pd None experts: 1 depth: 2 ...
Starting pdm6_input.pd None experts: 2 depth: 1 ...
Starting pdm6_input.pd None experts: 2 depth: 2 ...
Starting pdm6_input.pd None experts: 3 depth: 1 ...
Starting pdm6_input.pd None experts: 3 depth: 2 ...
Starting pdm6_input.pd None experts: 4 depth: 1 ...
Starting pdm6_input.pd None experts: 4 depth: 2 ...
Starting pdm6_input.pd None experts: 5 depth: 1 ...
Starting pdm6_input

In [7]:
df_performance

,dataset,n_features,train,test,train_std,test_std,n_experts,depth,gate
0,generated6,2,0.641467,0.639400,0.001852,0.008586,1,1,feature_importance_lr_max
1,generated6,2,0.741067,0.738800,0.003441,0.012060,1,2,feature_importance_lr_max
2,generated6,2,0.741667,0.738800,0.003726,0.008514,2,1,feature_importance_lr_max
3,generated6,2,0.815800,0.811200,0.001783,0.006425,2,2,feature_importance_lr_max
4,generated6,2,0.754067,0.748600,0.026387,0.021963,3,1,feature_importance_lr_max
...,...,...,...,...,...,...,...,...,...
555,banknote,4,1.000000,0.997085,0.000000,0.002062,8,2,None
556,banknote,4,1.000000,0.997813,0.000000,0.001262,9,1,None
557,banknote,4,1.000000,0.997085,0.000000,0.002062,9,2,None
558,banknote,4,0.999514,0.997813,0.000486,0.001262,10,1,None


In [8]:
pickle.dump(df_performance, open("dataframes/ex2_df_n_experts.pd", "wb"))